In [1]:
!git clone https://github.com/wyf0912/ExposureDiffusion.git
!cd ExposureDiffusion

Cloning into 'ExposureDiffusion'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 89 (delta 19), reused 67 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 3.51 MiB | 6.48 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [2]:
!pip install scikit-video
!pip install rawpy
!pip install torchinterp1d
!pip install exifread
!pip install lmdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchinterp1d: filename=torchinterp1d-1.1-py3-none-any.whl size=4737 sha256=c661e529316eb191a59d61470b09e5f76a5d5f352aeeedc867e9f88f5fac0e9d
  Stored in directory: /root/.cache/pip/wheels/c1/4a/f1/874cae91ed1f43a96f15f78e2f5f4ce275224a5661fa08360e
Successfully built torchinterp1d
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 6.4 MB/s eta 0:00:00


In [3]:
import sys
sys.path.append('/kaggle/working/ExposureDiffusion')

In [4]:
term_width = 80
with open('/kaggle/working/ExposureDiffusion/util/util.py', 'r') as f:
    lines = f.readlines()
lines[198] = "term_width = 80  # Hardcoded default\n"
lines[199] = "\n"  # Replace the next line with a blank
with open('/kaggle/working/ExposureDiffusion/util/util.py', 'w') as f:
    f.writelines(lines)

with open('/kaggle/working/ExposureDiffusion/dataset/torchdata.py', 'r') as f:
    lines = f.readlines()
# Replace the import
lines[3] = "# from torch._utils import _accumulate  # Removed for PyTorch 2.0+\n"
# Add a fallback if needed (check usage first)
if "_accumulate" in ''.join(lines):
    lines.insert(4, "from itertools import accumulate  # Fallback for _accumulate\n")
with open('/kaggle/working/ExposureDiffusion/dataset/torchdata.py', 'w') as f:
    f.writelines(lines)

In [5]:
import os
import torch
import noise
import dataset
import numpy as np
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import dataset.sid_dataset as datasets
import dataset.lmdb_dataset as lmdb_dataset
import util.util as util
import util.index as index

from PIL import Image
from os.path import join
from functools import partial
from skimage.metrics import structural_similarity, peak_signal_noise_ratio
from skvideo.measure import strred

from engine import Engine
from util import process
from util.index import quality_assess
from options.eld.base_options import BaseOptions
from options.eld.train_options import TrainOptions
from dataset.sid_dataset import worker_init_fn
from models.ELD_model import postprocess_bayer_v2, postprocess_bayer, postprocess_xtrans 
from models.ELD_model_iter import ELDModelIter
from models.ELD_model import ELDModelBase, tensor2im

In [6]:
# Change to the repo directory
os.chdir('/kaggle/working/ExposureDiffusion/')

opt = BaseOptions()

sys.argv = ['test_ELD.py']  # Dummy script name
opt = BaseOptions().parse()  
opt.nThreads = 4

------------ Options -------------
adaptive_res_and_x0: False
channels: 4
checkpoints_dir: ./checkpoints
chop: False
concat_origin: False
crf: False
debug: False
epoch: 200
gpu_ids: [0]
gt_wb: False
include: None
isTrain: False
iter_num: 2
model: eld_model
model_path: None
nThreads: 8
name: None
netG: unet
no_log: False
no_verbose: False
resid: False
resume: False
resume_epoch: None
seed: 2018
serial_batches: False
stage_eval: raw
stage_in: raw
stage_out: raw
with_photon: False
-------------- End ----------------


In [7]:
opt.model_path = '/kaggle/input/sid-pgru/pytorch/default/1/sid_PGru.pt'

# Create checkpoints directory
checkpoints_dir = "/kaggle/working/ExposureDiffusion/checkpoints/ELD_test"
os.makedirs(checkpoints_dir, exist_ok=True)  # Creates parent dirs if missing

# Enable CUDA optimizations
cudnn.benchmark = True

In [8]:
"""Main Loop"""
engine = Engine(opt)

databasedir = '/kaggle/input/aml-eld/ELD/ELD'
method = opt.name
scenes = list(range(1, 10+1))
cameras = ['CanonEOS5D4', 'CanonEOS70D', 'CanonEOS700D', 'NikonD850', 'SonyA7S2']     
suffixes = ['.CR2', '.CR2', '.CR2', '.nef', '.ARW']


if opt.include is not None:
    cameras = cameras[opt.include:opt.include+1]
    suffixes = suffixes[opt.include:opt.include+1]
else:
    cameras = ['CanonEOS70D', 'CanonEOS700D', 'NikonD850', 'SonyA7S2']     
    suffixes = ['.CR2', '.CR2', '.nef', '.ARW']


# img_ids_set = [[4, 9, 14]]
img_ids_set = [[4, 9, 14], [5, 10, 15]]
# for scene in scenes:
ratio = [100, 200] # [100, 200]

--------------------- Model ---------------------
UNetSeeInDark(
  (conv1_1): Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool4): MaxPool2d(kernel_size=2, str

## Doing this as in the repo a different function with the same name is defined, its getting wrongly called 
#### forward(): models/ELD_model_iter.py -> forward(): models/ELD_model.py 

In [9]:
def eval(self, data, savedir=None, suffix=None, correct=False, crop=True, frame_id=None, iter_num=0, old_diffusion=False):
        # only the 1st input of the whole minibatch would be evaluated
        self._eval()
        self.set_input(data, 'eval')

        # if self.data_name is not None and savedir is not None:
        #     name = os.path.splitext(os.path.basename(self.data_name[0]))[0]
        #     if not os.path.exists(join(savedir, name)):
        #         os.makedirs(join(savedir, name))
            
        #     for fn in os.listdir(join(savedir, name)):                
        #         if fnmatch.fnmatch(fn, '*{}_*'.format(self.opt.name)):
        #             return {}

        with torch.no_grad():
            ### evaluate center region to avoid fixed pattern noise
            cropx = 512; cropy = 512
            if crop:
                self.target = util.crop_center(self.target, cropx, cropy)
                self.input = util.crop_center(self.input, cropx, cropy)
            if not old_diffusion:
                if not self.opt.chop:
                    self.target, para = util.auto_padding(self.target, scale=16)
                    self.input, para = util.auto_padding(self.input, scale=16)
                output_list = ELDModelIter.forward(self, iter_num=iter_num)
                self.output = output_list[0]
            else:
                ABLATION_NUM = 0
                num_steps = 3
                if ABLATION_NUM == 0:
                    def to_photon(x, ratio, K):
                        # x: 0-1 image
                        return x/ratio/K*15583
                    def to_image(x, ratio_next, target_ratio, K):
                        return x/ratio_next*target_ratio*K/15583
                    ratio = int(self.ratio)
                    ratio_list = [1,50, 100]
                    # ratio_list = [1,2, 3]
                    # ratio_list =sorted(set(np.logspace(np.log10(1),np.log10(300),num_steps).astype(int)))
                    # ratio_list =[1,3]
                    # ratio_list =sorted(set(np.logspace(np.log10(1),np.log10(3),10).astype(float)))
                    for i in range(len(ratio_list)-1):
                        output_list = self.forward()
                        self.output = output_list[0]
                        ratio_current, ratio_next = ratio_list[i], ratio_list[i+1]
                        self.input = to_image(to_photon(self.input.clamp(0,1), ratio/ratio_current, self.K) + torch.poisson(to_photon(output.clamp(0,1), ratio/(ratio_next-ratio_current), self.K)), ratio_next, ratio, self.K)
                        
                        # self.input = to_image(to_photon(self.input, ratio, self.K)*ratio_current + to_photon(output, ratio, self.K)*(ratio_next-ratio_current), ratio_next, ratio, self.K)
                    output_list = self.forward()
                    self.output = output[0]
            #     elif ABLATION_NUM == 1:
            #         for i in range(num_steps):
            #             self.input = self.forward()
            #         self.output=self.input
                    
            #     elif ABLATION_NUM == 2:
            #         original_input = self.input
            #         for i in range(1,num_steps+1):
            #             self.output = self.forward()
            #             self.input = self.output * i/num_steps + (num_steps-i)/num_steps*original_input

            if not self.opt.chop:
                self.input = F.pad(self.input, para)
                self.target = F.pad(self.target, para)
                self.output = F.pad(self.output, para)
            if correct:
                self.output = self.corrector(self.output, self.target)
            
            if self.opt.stage_out == 'raw' and self.opt.stage_eval == 'srgb':
                target = postprocess_bayer_v2(self.rawpath, self.target)
                output = postprocess_bayer_v2(self.rawpath, self.output)
                input = postprocess_bayer_v2(self.rawpath, self.input)
            else:
                output = self.output
                target = self.target
                input = self.input

            output = tensor2im(output)
            target = tensor2im(target)   
            input = tensor2im(input)

            if target.shape[0] != output.shape[0]:
                target = np.repeat(target, output.shape[0], axis=0)

            res = index.quality_assess(output, target, data_range=255)
            res_in = index.quality_assess(input, target, data_range=255)  

            if savedir is not None and not crop:
                ## raw postprocessing
                if self.rawpath:
                    if self.cfa == 'bayer':
                        output = postprocess_bayer(self.rawpath, self.output)
                        target = postprocess_bayer(self.rawpath, self.target)
                        input = postprocess_bayer(self.rawpath, self.input)

                        # target = tensor2im(postprocess_bayer_v2(self.rawpath, self.target))
                        # output = tensor2im(postprocess_bayer_v2(self.rawpath, self.output))
                        # input = tensor2im(postprocess_bayer_v2(self.rawpath, self.input))

                    elif self.cfa == 'xtrans':
                        output = postprocess_xtrans(self.rawpath, self.output)
                        target = postprocess_xtrans(self.rawpath, self.target)
                        input = postprocess_xtrans(self.rawpath, self.input)
                    else:
                        raise NotImplementedError

                if self.data_name is not None:
                    if "ELD" in self.data_name[0]:
                        name = "_".join(self.data_name[0].split("/")[-2:]).split(".")[0]
                    else:
                        name = os.path.splitext(os.path.basename(self.data_name[0]))[0]

                    if not os.path.exists(join(savedir, name)):
                        os.makedirs(join(savedir, name))

                    if frame_id is not None:
                        if not os.path.exists(join(savedir, name, self.opt.name)):
                            os.makedirs(join(savedir, name, self.opt.name))

                        if not os.path.exists(join(savedir, name, 'input')):
                            os.makedirs(join(savedir, name, 'input'))                            

                        Image.fromarray(output.astype(np.uint8)).save(join(savedir, name, self.opt.name, '{}_{:.2f}.png'.format(frame_id, res['PSNR'])))

                        if not os.path.exists(join(savedir, name, 'input', '{}_{:.2f}.png'.format(frame_id, res_in['PSNR']))):
                            Image.fromarray(input.astype(np.uint8)).save(join(savedir, name, 'input', '{}_{:.2f}.png'.format(frame_id, res_in['PSNR'])))

                        if not os.path.exists(join(savedir, name, 'label')):
                            os.makedirs(join(savedir, name, 'label'))   

                        if not os.path.exists(join(savedir, name, 'label', '{}.png'.format(frame_id))):
                            Image.fromarray(target.astype(np.uint8)).save(join(savedir, name, 'label', '{}.png'.format(frame_id)))
                    else:
                        if suffix is not None:
                            Image.fromarray(output.astype(np.uint8)).save(join(savedir, name,'{}_{:.1f}_{}.png'.format(self.opt.name, res['PSNR'], suffix)))
                            # Image.fromarray(output.astype(np.uint8)).save(join(savedir, name,'{}_{:.1f}_{}.jpg'.format(self.opt.name, res['PSNR'], suffix)), optimize=True, quality=90)
                            Image.fromarray(input.astype(np.uint8)).save(join(savedir, name, 'm_input_{}.png'.format(suffix)))
                            # Image.fromarray(input.astype(np.uint8)).save(join(savedir, name, 'm_input_{}.jpg'.format(suffix)), optimize=True, quality=90)
                        else:
                            # Image.fromarray(output.astype(np.uint8)).save(join(savedir, name, '{}_{:.1f}.jpg'.format(self.opt.name, res['PSNR'])), optimize=True, quality=90)
                            Image.fromarray(output.astype(np.uint8)).save(join(savedir, name, '{}_{:.2f}_{:.2f}.png'.format(self.opt.model_path.split("/")[-2], res['PSNR'], res['SSIM'])))
                            # Image.fromarray(output.astype(np.uint8)).save(join(savedir, name, '{}.png'.format(self.opt.name)))
                            Image.fromarray(input.astype(np.uint8)).save(join(savedir, name, 'm_input_{:.2f}_{:.2f}.png'.format(res_in['PSNR'], res_in['SSIM'])))
                            # Image.fromarray(input.astype(np.uint8)).save(join(savedir, name, 'm_input.jpg'), optimize=True, quality=90)
                        
                        Image.fromarray(target.astype(np.uint8)).save(join(savedir, name, 't_label.png'))
                        # Image.fromarray(target.astype(np.uint8)).save(join(savedir, name, 't_label.jpg'), optimize=True, quality=90)

            return res

In [10]:
ELDModelBase.eval = eval

In [11]:
def quality_assess(X, Y, data_range=255):
    # Y: correct; X: estimate
    if X.ndim == 3:  # image
        psnr = peak_signal_noise_ratio(Y, X, data_range=data_range)
        ssim = structural_similarity(Y, X, data_range=data_range, channel_axis=-1)
        return {'PSNR':psnr, 'SSIM': ssim}

    elif X.ndim == 4:  # video clip
        vpsnr = np.mean(compare_psnr_video(Y/data_range*255, X/data_range*255))
        vssim = np.mean(compare_ssim_video(Y/data_range*255, X/data_range*255))

        if X.shape[0] != 1:
            _, _strred, _strredsn = strred(raw2gray(Y)/data_range, raw2gray(X)/data_range)
        else:
            _strred = 0
            _strredsn = 0

        return {'PSNR': vpsnr, 'SSIM': vssim, 'STRRED': _strred, 'STRREDSN':_strredsn}
    else:
        raise NotImplementedError

In [12]:
index.quality_assess = quality_assess

In [13]:
for i, img_ids in enumerate(img_ids_set):
    # if ratio[i]==100:
    #     continue
    print(img_ids)
    # eval_datasets = [datasets.ELDEvalDataset(databasedir, camera_suffix, scenes=[scene], img_ids=img_ids) for camera_suffix in zip(cameras, suffixes)]
    noise_model = noise.NoiseModel(model="P+g", include=opt.include)
    eval_datasets = [datasets.ELDEvalDataset(databasedir, camera_suffix, noise_model,scenes=scenes, img_ids=img_ids) for camera_suffix in zip(cameras, suffixes)]

    eval_dataloaders = [torch.utils.data.DataLoader(
        eval_dataset, batch_size=1, shuffle=False,
        num_workers=opt.nThreads, pin_memory=True) for eval_dataset in eval_datasets]

    # cameras = ['CanonEOS5D4', 'HuaweiHonor10']
    psnrs = []
    ssims = []
    for camera, dataloader in zip(cameras, eval_dataloaders):
        print('\n\n')
        print('Eval camera {}'.format(camera))

        print()
        # Get one batch from the dataloader
        data_iter = iter(dataloader)
        batch = next(data_iter)
        
        # Print the keys (tells you what kind of data is inside)
        print("Batch keys:", batch.keys())
        
        # Print the shape/type of each item
        for key, value in batch.items():
            if hasattr(value, 'shape'):
                print(f"{key}: shape = {value.shape}")
            else:
                print(f"{key}: type = {type(value)}, value = {value}")

        print()
        
        # we evaluate PSNR/SSIM on full size images
        crop = False
        savedir = f"images/{opt.model_path.split('/')[-2]}/{ratio[i]}"
        res = engine.eval(dataloader, dataset_name='eld_eval_{}'.format(camera), correct=True, crop=crop, savedir=savedir) 
        
        psnrs.append(res['PSNR'])
        ssims.append(res['SSIM'])
    print(f"Done with {i} img_ids_set")

[4, 9, 14]
[i] NoiseModel with camera_params/release
[i] cameras: ['CanonEOS5D4', 'CanonEOS70D', 'CanonEOS700D', 'NikonD850', 'SonyA7S2']
[i] using noise model P+g



Eval camera CanonEOS70D

Batch keys: dict_keys(['input', 'target', 'fn', 'rawpath', 'ratio', 'K'])
input: shape = torch.Size([1, 4, 1835, 2748])
target: shape = torch.Size([1, 4, 1835, 2748])
fn: type = <class 'list'>, value = ['/kaggle/input/aml-eld/ELD/ELD/CanonEOS70D/scene-1/IMG_0004.CR2']
rawpath: type = <class 'list'>, value = ['/kaggle/input/aml-eld/ELD/ELD/CanonEOS70D/scene-1/IMG_0006.CR2']
ratio: shape = torch.Size([1])
K: shape = torch.Size([1])

 [==============================================================>..]  Step: 17s811ms | Tot: 8m24s | PSNR: 25.0805 | SSIM: 0.5614 |  30/30 



Eval camera CanonEOS700D

Batch keys: dict_keys(['input', 'target', 'fn', 'rawpath', 'ratio', 'K'])
input: shape = torch.Size([1, 4, 1738, 2604])
target: shape = torch.Size([1, 4, 1

In [14]:
# opt = TrainOptions().parse()

# cudnn.benchmark = True

# evaldir = '/kaggle/input/aml-sld/Sony'
# traindir = './datasets/train'

# expo_ratio = [100, 300] # [100, 250, 300]
# read_expo_ratio = lambda x: float(x.split('_')[-1][:-5])

# # train_fns = dataset.read_paired_fns('./dataset/Sony_train.txt')
# # eval_fns = dataset.read_paired_fns('./dataset/Sony_val.txt')
# # test_fns = dataset.read_paired_fns('./dataset/Sony_test.txt')

# # eval_fns_list = [[fn for fn in eval_fns if min(int(read_expo_ratio(fn[1])/read_expo_ratio(fn[0])), 300)==ratio] for ratio in expo_ratio]
# # test_fns_list = [[fn for fn in test_fns if min(int(read_expo_ratio(fn[1])/read_expo_ratio(fn[0])), 300)==ratio] for ratio in expo_ratio]
# # eval_fns_list = [lst_1 + lst_2 for lst_1, lst_2 in zip(eval_fns_list, test_fns_list)]

# # evaluate 15 indoor scenes (but you can also evaluate the performance on the whole dataset)
# indoor_ids = dataset.read_paired_fns('./SID_Sony_15_paired.txt')
# eval_fns_list = [[(fn[0], fn[1]) for fn in indoor_ids if int(fn[2]) == ratio] for ratio in expo_ratio]

# cameras = ['CanonEOS5D4', 'CanonEOS70D', 'CanonEOS700D', 'NikonD850', 'SonyA7S2']
# noise_model = noise.NoiseModel(model=opt.noise, include=opt.include)

# repeat = 1 if opt.max_dataset_size is None else 1288 / opt.max_dataset_size
# print('[i] repeat:', repeat)

# CRF = None
# if opt.crf:
#     print('[i] enable CRF')
#     CRF = process.load_CRF()

# if opt.stage_out == 'srgb':
#     target_data = lmdb_dataset.LMDBDataset(join(traindir, 'SID_Sony_SRGB_CRF.db'))
# else:
#     target_data = lmdb_dataset.LMDBDataset(
#         join(traindir, 'SID_Sony_Raw.db'),
#         size=opt.max_dataset_size, repeat=repeat)
# if opt.stage_in == 'srgb':
#     input_data = datasets.ISPDataset(
#         lmdb_dataset.LMDBDataset(join(traindir, 'SID_Sony_Raw.db')),
#         noise_maker=noise_model, CRF=CRF)
# else:
#     ## Synthesizing noise on-the-fly by noise model    
#     input_data = datasets.SynDataset(
#         lmdb_dataset.LMDBDataset(join(traindir, 'SID_Sony_Raw.db')),
#         noise_maker=noise_model, num_burst=1,
#         size=opt.max_dataset_size, repeat=repeat, continuous_noise=opt.continuous_noise)

#     ## Noise generated offline    
#     # camera = cameras[opt.include]
#     # input_data = lmdb_dataset.LMDBDataset(
#     #     join(traindir, f'SID_Sony_syn_Raw_{camera}.db'),
#     #     size=opt.max_dataset_size, repeat=repeat)


# train_dataset =  datasets.ELDTrainDataset(target_dataset=target_data, input_datasets=[input_data])

# eval_datasets = [datasets.SIDDataset(evaldir, fns, noise_model, size=None, augment=False, memorize=False, stage_in=opt.stage_in, stage_out=opt.stage_out, gt_wb=opt.gt_wb, CRF=CRF) for fns in eval_fns_list]


# train_dataloader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=opt.batchSize, shuffle=True,
#     num_workers=opt.nThreads, pin_memory=True, worker_init_fn=worker_init_fn)


# eval_dataloaders = [torch.utils.data.DataLoader(
#     eval_dataset, batch_size=1, shuffle=False,
#     num_workers=0, pin_memory=True) for eval_dataset in eval_datasets]


# """Main Loop"""
# engine = Engine(opt)

# print('[i] using noise model {}'.format(opt.noise))

# # if opt.resume:
# # engine.eval(eval_dataloaders[0], dataset_name='sid_eval_100', correct=True)
# # engine.eval(eval_dataloaders[2], dataset_name='sid_eval_300', correct=True)

# engine.model.opt.save_epoch_freq = 100

# engine.set_learning_rate(opt.lr)
# while engine.epoch < opt.epoch:
#     np.random.seed()
#     if engine.epoch == opt.epoch//2:
#         engine.set_learning_rate(opt.lr/2)
#     if engine.epoch == int(opt.epoch*0.9):
#         engine.set_learning_rate(opt.lr/10)
    
#     engine.train(train_dataloader)
#     if engine.epoch % 10 == 0:
#         try:
#             print("Eval sid 100:")
#             engine.eval(eval_dataloaders[0], dataset_name='sid_eval_100', correct=True, iter_num=opt.iter_num)
#             print("Eval sid 300:")
#             engine.eval(eval_dataloaders[1], dataset_name='sid_eval_300', correct=True, iter_num=opt.iter_num)
#         except:
#             pass

* Lightning Memory-Mapped Database (LMDB) is an embedded transactional database in the form of a key-value store. LMDB is written in C with API bindings for several programming languages. LMDB stores arbitrary key/data pairs as byte arrays, has a range-based search capability, supports multiple data items for a single key and has a special mode for appending records (MDB_APPEND) without checking for consistency.[1] LMDB is not a relational database, it is strictly a key-value store like Berkeley DB and DBM.

In [15]:
# # Visualize your datasets (for debugging)
# # from os.path import join
# from options.eld.train_options import TrainOptions
# from dataset.lmdb_dataset import LMDBDataset
# # import torch
# # import dataset.sid_dataset as datasets
# import dataset
# # import cv2
# # import numpy as np
# # import noise


# opt = TrainOptions().parse()

# traindir = './data/Train'

# expo_ratio = [100, 250, 300]
# read_expo_ratio = lambda x: float(x.split('_')[-1][:-5])

# train_fns = dataset.read_paired_fns('/kaggle/input/aml-eld/Sony_data/Sony_train.txt')
# eval_fns = dataset.read_paired_fns('/kaggle/input/aml-eld/Sony_data/Sony_val.txt')
# test_fns = dataset.read_paired_fns('/kaggle/input/aml-eld/Sony_data/Sony_test.txt')

# eval_fns_list = [[fn for fn in eval_fns if min(int(read_expo_ratio(fn[1])/read_expo_ratio(fn[0])), 300)==ratio] for ratio in expo_ratio]
# eval_fns_list = [lst[-5:] for lst in eval_fns_list]
# test_fns_list = [[fn for fn in test_fns if min(int(read_expo_ratio(fn[1])/read_expo_ratio(fn[0])), 300)==ratio and int(fn[0][:5]) not in [10192, 10199, 10203]] for ratio in expo_ratio]
# test_fns_list = [lst[-10:] for lst in test_fns_list]
# eval_fns_list = [lst_1 + lst_2 for lst_1, lst_2 in zip(eval_fns_list, test_fns_list)]

# noise_model = noise.NoiseModel(model=opt.noise, include=opt.include, exclude=None)

# repeat = 1 if opt.max_dataset_size is None else 1288 // opt.max_dataset_size


# # target_data = LMDBDataset(join(traindir, 'SID_Sony_SRGB_CRF.db'))
# # input_data = LMDBDataset(join(traindir, 'SID_Sony_Raw.db'))

# # target_data = LMDBDataset(join(traindir, 'SID_Sony_target_Raw.db'))
# # input_data = LMDBDataset(join(traindir, 'SID_Sony_input_Raw.db'))

# target_data = LMDBDataset(
#     join(traindir, 'SID_Sony_Raw.db'), 
#     size=opt.max_dataset_size, repeat=repeat)

# # input_data = datasets.SynDataset(
# #     LMDBDataset(join(traindir, 'SID_Sony_Raw.db')),
# #     noise_maker=noise_model,
# #     size=opt.max_dataset_size, repeat=repeat
# # )

# # input_data = datasets.ISPDataset(
# #     LMDBDataset(join(traindir, 'SID_Sony_Raw.db')),
# #     noise_maker=None,    
# # )

# camera = 'NikonD850'
# input_data = LMDBDataset(
#     join(traindir, f'SID_Sony_syn_Raw_{camera}.db'),
#     size=opt.max_dataset_size, repeat=repeat)


# train_dataset = datasets.ELDTrainDataset(target_dataset=target_data, input_datasets=[input_data], size=10, augment=False)


# train_dataloader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=opt.batchSize, shuffle=True,
#     num_workers=0, pin_memory=True, worker_init_fn=datasets.worker_init_fn)


# """Main Loop"""
# from models.ELD_model import tensor2im

# for dataset in train_dataloader:
#     np.random.seed()

#     input, target = dataset['input'], dataset['target']

#     target_image = tensor2im(target, visualize=True)
#     input_image = tensor2im(input, visualize=True)

#     display = np.concatenate([input_image[:,:,::-1], target_image[:,:,::-1]], axis=1).astype(np.uint8)
    
#     cv2.imshow('display', display)

#     # cv2.waitKey()